In [2]:
# all imports
import base64
import datetime
from pprint import pprint
from networkx import Graph, DiGraph, simple_cycles
from bs4 import BeautifulSoup
import requests
import pandas as pd
import pickle
from IPython.display import display, HTML
import numpy as np
import scipy.stats
import matplotlib.pyplot as plt
import sklearn
from sklearn import preprocessing
from sklearn.preprocessing import PolynomialFeatures,StandardScaler,PolynomialFeatures
from sklearn.linear_model import Ridge
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split,KFold,RepeatedKFold
from sklearn.metrics import r2_score,mean_absolute_percentage_error
from scipy.stats import spearmanr
from numpy import mean,std
import seaborn as sns
from sklearn.model_selection import StratifiedKFold

from github import Github
from tabulate import tabulate
import random
from random import choice, choices, shuffle, sample
#pd.set_option("display.max_rows", None, "display.max_columns", None)
import networkx as nx
!pip install table-evaluator
from table_evaluator import load_data, TableEvaluator
from sklearn.decomposition import PCA

In [ ]:
repo_dic={'Android-Universal-Image-Loader':'nostra13/Android-Universal-Image-Loader',
         'antlr':'antlr/antlr4',
         'BroadleafCommerce':'BroadleafCommerce/BroadleafCommerce',
         'hazelcast':'hazelcast/hazelcast',
         'junit':'junit-team/junit',
         'mapdb':'jankotek/mapdb',
         'mcMMO':'mcMMO-Dev/mcMMO',
         'nasa_mct':'nasa/mct',
         'neo4j':'neo4j/neo4j',
         'netty':'netty/netty',
         'orientdb':'orientechnologies/orientdb',
         'titan':'thinkaurelius/titan'}

In [ ]:
for key, value in repo_dic.items():

    df=pd.read_csv('data/commits/'+key+'.csv',index_col=0)
    
    #remove committers which is null values 
    df=df.dropna(subset=['committer']).reset_index(drop=True)

    lst=df.SHA.unique().tolist()

    #remove files changes greater than 200
    df=df[df['files changed']<200].reset_index(drop=True) 

    #group commits based on 3 months
    df['date'] = df['date'].apply(lambda _: datetime.datetime.strptime(_,'%Y-%m-%d %H:%M:%S')) 
    df=df.sort_values(by='date',ascending=False).reset_index(drop=True)
    #bug dataset collection end date
    start_time = '6/1/15'     
    # Define dates as datetime objects
    start_time = datetime.datetime.strptime(start_time, '%m/%d/%y')
    #start_time
    time_change = datetime.timedelta(weeks=13)
    #time_change
    time_lst=[]
    time=start_time
    time_lst.append(time)
    for s_date in df.date:
        if time<df.iloc[-1].date:
            break
        time = time-time_change
        time_lst.append(time)
 
    #create commit groups
    commitgroups=[]
    for value in range(len(time_lst)-1):
        df_interval=df [ (df.date<time_lst[value]) & (df.date>time_lst[value+1]) ]
        commitgroups.extend(df_interval.groupby('filename')['committer'].unique().tolist())

    #construct developer networks
    %matplotlib inline
    G_symmetric1 = nx.Graph()
    for group in commitgroups:
        test_list = group
        res = [(a, b) for idx, a in enumerate(test_list) for b in test_list[idx + 1:]] 
        for x in res:    
            G_symmetric1.add_edge(x[0],x[1])

    #save commit groups characterized to DSN
    a_file = open("data/usecase_data/commit_groups/commitGroups_"+key+".pkl", "wb")
    pickle.dump(commitgroups, a_file)
    a_file.close()


### Model_training

In [ ]:
for key, value in repo_dic.items():
    
    df=pd.read_csv('data/commits/'+key+'.csv',index_col=0)

    a_file = open("data/usecase_data/commit_groups/commitGroups_"+key+".pkl", "rb")
    commit_groups = pickle.load(a_file)

    # relevant repository bugdataset 
    df_bugdata=pd.read_csv('data/usecase_data/bug_data/'+key+'_BugData.csv',index_col=0)
    df_bugdata=df_bugdata[['LongName','Number of bugs']]
    df_bugdata.columns=['filename','no_of_bugs']
    df_bugdata=df_bugdata.groupby(by='filename').sum().reset_index()

    #preprocessing df
    df['date'] = df['date'].apply(lambda _: datetime.datetime.strptime(_,'%Y-%m-%d %H:%M:%S'))
    df=df.sort_values(by='date',ascending=False).reset_index(drop=True)
    #remove files after bug dataset end date 6/1/15
    start_time = '6/1/15'     
    # Define dates as datetime objects
    start_time = datetime.datetime.strptime(start_time, '%m/%d/%y')
    df = df [(df.date<start_time)]
    df = df.reset_index(drop=True)

    #Dataframe for committers groups for files
    df_file_committers=pd.DataFrame(df.groupby('filename')['committer'].unique()).reset_index()

    # define graph
    %matplotlib inline
    G_symmetric = nx.Graph()
    for group in commit_groups:
        test_list = group
        res = [(a, b) for idx, a in enumerate(test_list) for b in test_list[idx + 1:]] 
        for x in res:    
            G_symmetric.add_edge(x[0],x[1])
    
    #find file-based network centrality metrics
    df_file_committers['degree_sum']=0.0
    df_file_committers['betweenness_sum']=0.0
    df_file_committers['closeness_sum']=0.0
    df_file_committers['clustering_sum']=0.0
    df_file_committers['eigenvector_sum']=0.0
    df_file_committers['pagerank_sum']=0.0
    df_file_committers['hubs_sum']=0.0
    df_file_committers['Number of Hubs']=0
    
    hub_limit=int(sorted(dict(nx.degree(G_symmetric)).values(),reverse=True)[0]*0.5) 
    hubs_dict=nx.hits(G_symmetric)[0]
    pagerank_dict=nx.pagerank(G_symmetric)
    eigenvector_dict=nx.eigenvector_centrality(G_symmetric)
    clustering_dict=nx.clustering(G_symmetric)
    betweenness_dict=nx.betweenness_centrality(G_symmetric)
    closeness_dict=nx.closeness_centrality(G_symmetric)
    
    for index in df_file_committers.index:
        degree_sum=0.0
        betweenness_sum=0.0
        closeness_sum=0.0
        clustering_sum=0.0
        eigenvector_sum=0.0
        pagerank_sum=0.0
        hubs_sum=0.0
        count_hubs=0
        for author in df_file_committers['committer'][index]:
            try:    
                degree_sum+=G_symmetric.degree(author)
                betweenness_sum+=betweenness_dict[author]
                closeness_sum+=closeness_dict[author] 
                clustering_sum+=clustering_dict[author]
                eigenvector_sum+=eigenvector_dict[author]
                pagerank_sum+=pagerank_dict[author]
                hubs_sum+=hubs_dict[author]
                if G_symmetric.degree(author)>hub_limit:
                    count_hubs+=1
            except:
                continue
                
        # take average of all centrality measures
        df_file_committers['degree_sum'][index]=degree_sum/len(df_file_committers['committer'][index])
        df_file_committers['betweenness_sum'][index]=betweenness_sum/len(df_file_committers['committer'][index])
        df_file_committers['closeness_sum'][index]=closeness_sum/len(df_file_committers['committer'][index])
        df_file_committers['clustering_sum'][index]=clustering_sum/len(df_file_committers['committer'][index])
        df_file_committers['eigenvector_sum'][index]=eigenvector_sum/len(df_file_committers['committer'][index])
        df_file_committers['pagerank_sum'][index]=pagerank_sum/len(df_file_committers['committer'][index])
        df_file_committers['hubs_sum'][index]=hubs_sum/len(df_file_committers['committer'][index])
        df_file_committers['Number of Hubs'][index]=count_hubs
    
    #dataframe for number of times each file is touched
    df_file_updates=pd.DataFrame(df['filename'].value_counts()).reset_index().rename(columns={'filename':'Updates','index':'filename'})

    #Dataframe for network metrics
    df_centrality_measures=pd.merge(df_file_committers,df_file_updates,right_on='filename',left_on='filename')
    df_centrality_measures['Developers']=df_centrality_measures['committer'].apply(lambda x: len(x))
    
    #Dataframe for code churn in each file
    df_churn=pd.DataFrame(df.groupby('filename')['file changes'].sum()).reset_index().rename(columns={'file changes':'code churn'})

    #Dataframe including code churn metric
    df_centrality_churn=pd.merge(df_centrality_measures,df_churn,right_on='filename',left_on='filename')

    #training dataframe with Bug data included
    df_trainData=pd.merge(df_centrality_churn,df_bugdata,right_on='filename',left_on='filename') 

    #save training data
    df_trainData.to_csv('data/usecase_data/train_data/'+key+'_TrainData.csv')
    

In [ ]:
# load train data for different projects
df1=pd.read_csv('data/usecase_data/train_data/Android-Universal-Image-Loader_TrainData.csv',index_col=0)
df2=pd.read_csv('data/usecase_data/train_data/antlr_TrainData.csv',index_col=0)
df3=pd.read_csv('data/usecase_data/train_data/BroadleafCommerce_TrainData.csv',index_col=0)
df5=pd.read_csv('data/usecase_data/train_data/hazelcast_TrainData.csv',index_col=0)
df14=pd.read_csv('data/usecase_data/train_data/junit_TrainData.csv',index_col=0)
df7=pd.read_csv('data/usecase_data/train_data/mapdb_TrainData.csv',index_col=0)
df8=pd.read_csv('data/usecase_data/train_data/mcMMO_TrainData.csv',index_col=0)
df9=pd.read_csv('data/usecase_data/train_data/nasa_mct_TrainData.csv',index_col=0)
df10=pd.read_csv('data/usecase_data/train_data/neo4j_TrainData.csv',index_col=0)
df11=pd.read_csv('data/usecase_data/train_data/netty_TrainData.csv',index_col=0)
df12=pd.read_csv('data/usecase_data/train_data/orientdb_TrainData.csv',index_col=0)
df13=pd.read_csv('data/usecase_data/train_data/titan_TrainData.csv',index_col=0)

In [ ]:
# non average centrality measures
data_lst = [df1,df2,df3,df5,df14,df7,df8,df9,df10,df11,df12,df13] 
for data in data_lst:
    data['degree_sum']=data['degree_sum']*data['Developers']
    data['betweenness_sum']=data['betweenness_sum']*data['Developers']
    data['closeness_sum']=data['closeness_sum']*data['Developers']
    data['clustering_sum']=data['clustering_sum']*data['Developers']
    data['eigenvector_sum']=data['eigenvector_sum']*data['Developers']
    data['pagerank_sum']=data['pagerank_sum']*data['Developers']
    data['hubs_sum']=data['hubs_sum']*data['Developers']
    data['len_developers']=data['Developers']
    #drop zero rows
    data.drop(data[(data.degree_sum==0) & (data.betweenness_sum==0) & (data.closeness_sum==0)].index,inplace=True)

In [ ]:
#normalize predictor metrics
cols_to_norm = ['degree_sum','betweenness_sum','closeness_sum','Developers','clustering_sum', 'eigenvector_sum', 
                'pagerank_sum','hubs_sum','Number of Hubs']
data_lst = [df1,df2,df3,df5,df14,df7,df8,df9,df10,df11,df12,df13] 
for data in data_lst:
    data[cols_to_norm] = data[cols_to_norm].apply(lambda x: (x - x.min()) / (x.max() - x.min()))

#find probability of bug occurrence
cols_to_norm = ['no_of_bugs']
data_lst = [df1,df2,df3,df5,df14,df7,df8,df9,df10,df11,df12,df13] 
for data in data_lst:
    data[cols_to_norm] = data[cols_to_norm].apply(lambda x: (x) / (x.sum()))


In [ ]:
#merge data
df=df1.append([df2,df3,df5,df14,df7,df8,df9,df10,df11,df12,df13]) 
#select required columns 
df=df.reset_index(drop=True)
df=df[['degree_sum','betweenness_sum','closeness_sum','Developers','clustering_sum', 'eigenvector_sum', 
                'pagerank_sum','hubs_sum','Number of Hubs','no_of_bugs','committer','len_developers','filename']]

df.rename(columns={'no_of_bugs':'bug_probability'},inplace=True)

In [ ]:
#undersample non-buggy data
df_zero = df[df.bug_probability==0]
df_nonzero = df[df.bug_probability!=0]

#drop duplicates
df_zero.drop_duplicates(inplace=True)
df_nonzero.drop_duplicates(inplace=True)

perc = df_nonzero.shape[0]/df_zero.shape[0]         
df_zero_sampled=df_zero.sample(frac=perc,random_state=2550)  

#equalized portions
df_sampled = df_nonzero.append(df_zero_sampled).reset_index(drop=True)


In [ ]:
#drop duplicates
df_model=df_sampled       
df_model.drop_duplicates(inplace=True)   


In [ ]:
# log transformations

df_model.iloc[:,:-3]=df_model.iloc[:,:-3]+0.000001 

df_model['degree_sum'] = np.log(df_model['degree_sum'])
df_model['Developers'] = np.log(df_model['Developers'])
df_model['closeness_sum'] = np.log(df_model['closeness_sum'])
df_model['Number of Hubs'] = np.log(df_model['Number of Hubs'])
df_model['betweenness_sum'] = np.log(df_model['betweenness_sum'])
df_model['clustering_sum'] = np.log(df_model['clustering_sum'])
df_model['eigenvector_sum'] = np.log(df_model['eigenvector_sum'])
df_model['pagerank_sum'] = np.log(df_model['pagerank_sum'])
df_model['hubs_sum'] = np.log(df_model['hubs_sum'])
df_model['bug_probability'] = np.log(df_model['bug_probability'])

# shuffle data
df_model=df_model.sample(frac=1,random_state=2550)      
#df_model.to_csv('data/usecase_data/df_model.csv')

In [ ]:
# CROSS VALIDATION

n_splits=5
kf5 = KFold(n_splits=n_splits) 

spearman_sum=0
MAPE_sum=0

for train_index, test_index in kf5.split(df_model):
    X_train = df_model.iloc[train_index].iloc[:, [0,1,2,3,7]]
    X_test = df_model.iloc[test_index].iloc[:, [0,1,2,3,7]]
    y_train = df_model.iloc[train_index].loc[:,'bug_probability']
    y_test =  df_model.iloc[test_index].loc[:,'bug_probability']

    #Ridge regression
    model =  Ridge()
    #Training the model  
    model.fit(X_train, y_train)   
    predictions = model.predict(X_test)
    # calculate spearman's correlation
    coef, p = spearmanr(predictions, y_test)
    spearman_sum+=coef
    MAPE_sum+=mean_absolute_percentage_error(y_test, predictions)

# model    
X=df_model.iloc[:,[0,1,2,3,7]]
y=df_model.iloc[:,[9]]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2,random_state=2550)  
model = Ridge()                  
model.fit(X_train, y_train)
predictions = model.predict(X_test)

# calculate spearman's correlation
coef, p = spearmanr(predictions, y_test)